<center>
<font size='4'><b>Modelling landslide susceptibility of Cox's Bazar district and Rohingya camps in southeastern Bangladesh through machine learning</b></font><br><br>

    
</center>

In [2]:
import joblib
import os
import pandas as pd

#### Load estimators

In [2]:
knn = joblib.load(os.path.join('..','models','knn.pkl'))
lr = joblib.load(os.path.join('..','models','lr.pkl'))
svm = joblib.load(os.path.join('..','models','svm.pkl'))
mlp = joblib.load(os.path.join('..','models','mlp.pkl'))
dt = joblib.load(os.path.join('..','models','dt.pkl'))
rf = joblib.load(os.path.join('..','models','rf.pkl'))

#### Load data

In [3]:
df = pd.read_csv(os.path.join('..','data','Point_data.csv'))

In [4]:
df.shape

(1570994, 14)

In [5]:
preds = pd.DataFrame()

In [6]:
df.head(5)

,OBJECTID,pointid,Elevation,Aspect,Curvature,Geology,LULC_2020,NDVI,Road_dist,Slope,SoilTexture,SoilType,SPI,Stream_dist
0,1,1,75.0,302.195740,0.222222,3,2,0.289782,3062.352051,7.726209,3,1,-8.915490,591.692505
1,2,2,78.0,322.125031,0.555556,3,2,0.297374,3037.712891,5.779747,3,1,-9.248405,566.038879
2,3,3,77.0,36.253838,-0.222222,3,2,0.302728,3013.171143,4.122546,3,1,-9.449757,540.832703
3,4,4,74.0,79.824486,-0.000000,3,2,0.331275,2988.728760,8.085025,4,1,-1.098921,516.139526
4,5,5,52.0,40.364536,0.777778,3,2,0.327888,1958.979370,11.215478,4,1,-8.420445,120.000000


#### Slice relevant columns

In [7]:
preds['OBJECTID'] =  df['OBJECTID']
preds['pointid'] =  df['pointid']
df = df[df.columns[2:]]
df



,Elevation,Aspect,Curvature,Geology,LULC_2020,NDVI,Road_dist,Slope,SoilTexture,SoilType,SPI,Stream_dist
0,75.0,302.195740,0.222222,3,2,0.289782,3062.352051,7.726209,3,1,-8.915490,591.692505
1,78.0,322.125031,0.555556,3,2,0.297374,3037.712891,5.779747,3,1,-9.248405,566.038879
2,77.0,36.253838,-0.222222,3,2,0.302728,3013.171143,4.122546,3,1,-9.449757,540.832703
3,74.0,79.824486,-0.000000,3,2,0.331275,2988.728760,8.085025,4,1,-1.098921,516.139526
4,52.0,40.364536,0.777778,3,2,0.327888,1958.979370,11.215478,4,1,-8.420445,120.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1570989,5.0,315.000000,0.222222,9,2,0.242205,1103.086548,0.656380,3,1,-11.883472,684.105225
1570990,5.0,108.434952,0.333333,9,2,0.253525,1130.884644,1.348841,3,1,-11.162214,713.091858
1570991,5.0,288.434937,0.555556,9,0,0.201850,1181.862915,1.672718,3,1,-9.830180,750.000000
1570992,6.0,135.000000,0.777778,9,0,0.215794,1170.000000,0.984956,3,1,-11.883472,750.000000


In [8]:
df.shape

(1570994, 12)

In [9]:
df = df.drop(columns=['SoilTexture','SoilType'])

#### Rearrange columns

In [10]:
df = df[['Slope', 'Stream_dist', 'Road_dist', 'Elevation', 'Curvature', 'NDVI',
       'Aspect', 'SPI', 'Geology', 'LULC_2020']]

In [11]:
df

,Slope,Stream_dist,Road_dist,Elevation,Curvature,NDVI,Aspect,SPI,Geology,LULC_2020
0,7.726209,591.692505,3062.352051,75.0,0.222222,0.289782,302.195740,-8.915490,3,2
1,5.779747,566.038879,3037.712891,78.0,0.555556,0.297374,322.125031,-9.248405,3,2
2,4.122546,540.832703,3013.171143,77.0,-0.222222,0.302728,36.253838,-9.449757,3,2
3,8.085025,516.139526,2988.728760,74.0,-0.000000,0.331275,79.824486,-1.098921,3,2
4,11.215478,120.000000,1958.979370,52.0,0.777778,0.327888,40.364536,-8.420445,3,2
...,...,...,...,...,...,...,...,...,...,...
1570989,0.656380,684.105225,1103.086548,5.0,0.222222,0.242205,315.000000,-11.883472,9,2
1570990,1.348841,713.091858,1130.884644,5.0,0.333333,0.253525,108.434952,-11.162214,9,2
1570991,1.672718,750.000000,1181.862915,5.0,0.555556,0.201850,288.434937,-9.830180,9,0
1570992,0.984956,750.000000,1170.000000,6.0,0.777778,0.215794,135.000000,-11.883472,9,0


#### One hot encode categorical variables

In [12]:
from sklearn.preprocessing import OneHotEncoder

non_categorical_variables = ['Slope', 'Stream_dist', 'Road_dist', 'Elevation', 'Curvature', 'NDVI',
       'Aspect', 'SPI']

# Add numerical feature names to final feature name list
final_feature_names = []
final_feature_names += non_categorical_variables

# List of categorical feature names
categorical_features = ['Geology', 'LULC_2020']

# Copy numerical features
data = df[non_categorical_variables]

# Copy one-hot encoded categorical features
for i in categorical_features:
    encoded = OneHotEncoder(sparse=False).fit_transform(df[i].values.reshape(df.shape[0],1))
    cols = [i+'_'+str(j) for j in range (1, encoded.shape[1]+1)]
    data = pd.concat([data, pd.DataFrame(encoded, columns=cols)], axis = 1)




In [13]:
data

,Slope,Stream_dist,Road_dist,Elevation,Curvature,NDVI,Aspect,SPI,Geology_1,Geology_2,...,Geology_6,Geology_7,Geology_8,Geology_9,Geology_10,Geology_11,LULC_2020_1,LULC_2020_2,LULC_2020_3,LULC_2020_4
0,7.726209,591.692505,3062.352051,75.0,0.222222,0.289782,302.195740,-8.915490,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,5.779747,566.038879,3037.712891,78.0,0.555556,0.297374,322.125031,-9.248405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.122546,540.832703,3013.171143,77.0,-0.222222,0.302728,36.253838,-9.449757,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,8.085025,516.139526,2988.728760,74.0,-0.000000,0.331275,79.824486,-1.098921,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,11.215478,120.000000,1958.979370,52.0,0.777778,0.327888,40.364536,-8.420445,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570989,0.656380,684.105225,1103.086548,5.0,0.222222,0.242205,315.000000,-11.883472,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1570990,1.348841,713.091858,1130.884644,5.0,0.333333,0.253525,108.434952,-11.162214,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1570991,1.672718,750.000000,1181.862915,5.0,0.555556,0.201850,288.434937,-9.830180,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1570992,0.984956,750.000000,1170.000000,6.0,0.777778,0.215794,135.000000,-11.883472,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
data.columns

Index(['Slope', 'Stream_dist', 'Road_dist', 'Elevation', 'Curvature', 'NDVI',
       'Aspect', 'SPI', 'Geology_1', 'Geology_2', 'Geology_3', 'Geology_4',
       'Geology_5', 'Geology_6', 'Geology_7', 'Geology_8', 'Geology_9',
       'Geology_10', 'Geology_11', 'LULC_2020_1', 'LULC_2020_2', 'LULC_2020_3',
       'LULC_2020_4'],
      dtype='object')

#### Feature scaling

In [15]:
from sklearn.preprocessing import StandardScaler

# Stan
features = data.columns

# Separating out the features
X = data.loc[:, features].values

X = StandardScaler().fit_transform(X)

In [16]:
X.shape

(1570994, 23)

In [17]:
X

array([[ 0.50763558,  2.27558962,  0.18590672, ...,  0.62151169,
        -0.17479696, -0.42949923],
       [ 0.11111638,  2.13749294,  0.17661706, ...,  0.62151169,
        -0.17479696, -0.42949923],
       [-0.22647677,  2.00180493,  0.16736413, ...,  0.62151169,
        -0.17479696, -0.42949923],
       ...,
       [-0.725538  ,  3.12777871, -0.52309058, ..., -1.6089802 ,
        -0.17479696, -0.42949923],
       [-0.86564396,  3.12777871, -0.52756324, ..., -1.6089802 ,
        -0.17479696, -0.42949923],
       [-1.01866234,  3.84010873, -0.41226622, ...,  0.62151169,
        -0.17479696, -0.42949923]])

In [20]:
preds

,OBJECTID,pointid
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
1570989,1570990,1570990
1570990,1570991,1570991
1570991,1570992,1570992
1570992,1570993,1570993


In [22]:
preds['KNN'] = pd.Series(knn.predict_proba(X)[:,1])
preds['LR'] = pd.Series(lr.predict_proba(X)[:,1])
preds['MLP'] = pd.Series(mlp.predict_proba(X)[:,1])
preds['SVM'] = pd.Series(svm.predict_proba(X)[:,1])
preds['RF'] = pd.Series(rf.predict_proba(X)[:,1])
preds['DT'] = pd.Series(dt.predict_proba(X)[:,1])

In [23]:
preds.head(5)

,OBJECTID,pointid,KNN,LR,MLP,SVM,RF,DT
0,1,1,0.000000,0.994069,0.986894,0.999998,0.315385,0.0
1,2,2,0.000000,0.988444,0.962441,0.999983,0.305994,0.0
2,3,3,0.000000,0.988777,0.992876,0.999993,0.202434,0.0
3,4,4,0.196090,0.996711,0.992554,0.999999,0.362815,0.0
4,5,5,0.384963,0.986996,0.924938,0.999995,0.442361,0.0


In [25]:
preds.to_csv(os.path.join('..','data','predictions.csv'))

In [3]:
# preds = pd.read_csv(os.path.join('..','data','predictions.csv'))

We are choosing combinations of 2 models from a set of 5 models.

Number of combinations = $5\choose2$ = 10

Possible combinations:

1) RF-SVM

2) RF-MLP

3) RF-LR

4) RF-KNN

5) SVM-MLP

6) SVM-LR

7) SVM-KNN

8) MLP-LR

9) MLP-KNN

10) LR-KNN

In [4]:
combinations = ['RF-SVM','RF-MLP', 'RF-LR', 'RF-KNN','SVM-MLP','SVM-LR','SVM-KNN', 'MLP-LR', 'MLP-KNN', 'LR-KNN']

In [5]:
for combination in combinations:
    tokens = combination.split('-')
    preds [combination] = preds[tokens[0]] - preds[tokens[1]]

In [6]:
preds.head(5)

,Unnamed: 0,OBJECTID,pointid,KNN,LR,MLP,SVM,RF,DT,RF-SVM,RF-MLP,RF-LR,RF-KNN,SVM-MLP,SVM-LR,SVM-KNN,MLP-LR,MLP-KNN,LR-KNN
0,0,1,1,0.000000,0.994069,0.986894,0.999998,0.315385,0.0,-0.684612,-0.671509,-0.678684,0.315385,0.013103,0.005928,0.999998,-0.007175,0.986894,0.994069
1,1,2,2,0.000000,0.988444,0.962441,0.999983,0.305994,0.0,-0.693989,-0.656447,-0.682450,0.305994,0.037542,0.011539,0.999983,-0.026003,0.962441,0.988444
2,2,3,3,0.000000,0.988777,0.992876,0.999993,0.202434,0.0,-0.797559,-0.790442,-0.786343,0.202434,0.007117,0.011216,0.999993,0.004099,0.992876,0.988777
3,3,4,4,0.196090,0.996711,0.992554,0.999999,0.362815,0.0,-0.637184,-0.629739,-0.633896,0.166724,0.007445,0.003288,0.803909,-0.004157,0.796463,0.800621
4,4,5,5,0.384963,0.986996,0.924938,0.999995,0.442361,0.0,-0.557633,-0.482576,-0.544635,0.057399,0.075057,0.012998,0.615032,-0.062059,0.539975,0.602034


In [7]:
agreement = preds.drop(columns=['KNN','LR','MLP','SVM','RF','DT'])

In [8]:
agreement.head(5)

,Unnamed: 0,OBJECTID,pointid,RF-SVM,RF-MLP,RF-LR,RF-KNN,SVM-MLP,SVM-LR,SVM-KNN,MLP-LR,MLP-KNN,LR-KNN
0,0,1,1,-0.684612,-0.671509,-0.678684,0.315385,0.013103,0.005928,0.999998,-0.007175,0.986894,0.994069
1,1,2,2,-0.693989,-0.656447,-0.682450,0.305994,0.037542,0.011539,0.999983,-0.026003,0.962441,0.988444
2,2,3,3,-0.797559,-0.790442,-0.786343,0.202434,0.007117,0.011216,0.999993,0.004099,0.992876,0.988777
3,3,4,4,-0.637184,-0.629739,-0.633896,0.166724,0.007445,0.003288,0.803909,-0.004157,0.796463,0.800621
4,4,5,5,-0.557633,-0.482576,-0.544635,0.057399,0.075057,0.012998,0.615032,-0.062059,0.539975,0.602034


In [9]:
agreement.to_csv(os.path.join('..','data','agreement.csv'))